# Scrape CSD 
NOTE: you need a CSD licence to install and use the CSD API

In [12]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd

In [13]:
import ccdc
import ccdc.io
import ccdc.search
print("ccdc.__version__", ccdc.__version__)
print("ccdc.io.csd_directory()", ccdc.io.csd_directory())
print("ccdc.io.csd_version()", ccdc.io.csd_version())

subset_mofs_all = Path(ccdc.io.csd_directory()) / "subsets" / "MOF_subset.gcd"
print(f"Number of MOFs: {len(ccdc.io.EntryReader(str(subset_mofs_all)))}")

ccdc.__version__ 3.0.4
ccdc.io.csd_directory() /home/daniele/Programs/CCDC/CSD_2021/csd
ccdc.io.csd_version() 542
Number of MOFs: 105922


In [17]:
n_entries = len(ccdc.io.EntryReader(str(subset_mofs_all)))
rows = [{} for _ in range(n_entries)] # Pre allocate for speed
for i, entry in enumerate(tqdm(ccdc.io.EntryReader(str(subset_mofs_all)))): # Takes 2 minutes
    for col in ['identifier', 'ccdc_number','formula', 'has_disorder']:
        rows[i][col] = getattr(entry, col)
    rows[i]['publication_year'] = entry.publication.year
    doi = entry.publication.doi
    if doi: # can be None
        doi = doi.lower() # Lower caption to improve match in a later step
    rows[i]['publication_doi'] = doi 
    synonyms = list(entry.synonyms)
    rows[i]['synonyms_orig'] = synonyms
df = pd.DataFrame(rows) 

df.to_csv("data/step-00.csv", index=False) 
# step-00.csv contains all CSD entries of MOF subset: id, formula, has_disorder, pub year, doi, synonyms_orig

100%|██████████| 105922/105922 [01:54<00:00, 921.87it/s]


In [18]:
df.head(3)

,identifier,ccdc_number,formula,has_disorder,publication_year,publication_doi,synonyms_orig
0,ABACUF,1100034.0,(C6 H14 Ba2 Cu1 O16)n,False,1958,None,[]
1,ABACUF01,230290.0,(C6 H14 Ba2 Cu1 O16)n,False,2004,10.1016/j.molstruc.2004.03.051,[]
2,ABAFUH,1498688.0,(C9 H8 Cu2 O4)n,False,2016,10.1016/j.poly.2016.09.043,[]
